In [128]:
import numpy as np
N=100
kar = np.arange (N)*2

In [137]:
def generate_data (sig = 1e-3, A=0.1, phi_ofs=np.random.uniform(0,2*np.pi)):
    n = np.random.normal (0, 1/np.sqrt(2),N)+1j*np.random.normal(0,1/np.sqrt(2),N)
    s = A*np.exp(1j*(phi_ofs+kar*sig))
    return s+n

def estimate_phi (data):
    sum0 = np.sum(data)
    sum1 = np.sum(data*kar*1j)
    sum2 = np.sum(-data*kar**2)
    FD = np.real(sum0*np.conj(sum1))
    SD = np.real(sum2*np.conj(sum0)+sum1*np.conj(sum1))
    return FD/SD

def get_error (sig, A, phi_ofs=np.random.uniform(0,2*np.pi), Ngo=100000):
    res = [estimate_phi(generate_data(sig, A, sig)) for i in range(Ngo)]
    res = np.array(res)
    mean = res.mean()
    std = np.sqrt((res**2).mean()-sig**2)
    return mean, std

In [146]:
get_error(1e-3, 5.0,0, 100000)

(0.0010070129461629306, 0.00027583660556021027)